In [ ]:
import joblib
import matplotlib.pyplot as plt
import gempy as gp
import gempy_viewer as gpv
import numpy as np
import devito as dv
from examples.seismic import Model
from examples.seismic import TimeAxis
from examples.seismic import RickerSource
from examples.seismic import Receiver
from obspy import Stream, Trace
from devito import Eq, Operator, solve
import os, glob
from obspy import read
from matplotlib.patches import Rectangle, Patch

geo_data = joblib.load('modelo_gempy_topografia_25m_res_3.pkl')
fig = gpv.plot_2d(geo_data, direction='y', show_data=False, show_topography=True)

ax = fig.axes[0]  # Obtener el primer eje del gráfico
ax.set_aspect(2)  # Puedes usar un valor menor a 1 para estirar el eje Y (por ejemplo: 0.5, 0.3, etc.)

ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))

plt.tight_layout()
plt.show()

extent = geo_data.grid._dense_grid.extent
print("Extent:", extent)

res = geo_data.grid._dense_grid.resolution
print("Resolución:", res)

dx = (extent[1] - extent[0]) / res[0]
dy = (extent[3] - extent[2]) / res[1]
dz = (extent[5] - extent[4]) / res[2]

print(f"Espaciado: dx = {dx:.2f} m, dy = {dy:.2f} m, dz = {dz:.2f} m")

lith_vol = geo_data.solutions._raw_arrays.lith_block

lit_to_vp = {
    2: 1.800,
    3: 2.400,
    4: 2.600,
    5: 2.700,
    6: 2.500,
    7: 2.500,
    8: 2.700,
    9: 2.700,
    10: 2.600,
    11: 2.600,
    12: 3.800,
    13: 3.400,
    14: 3.800,
    15: 3.200,
    16: 3.900,
    17: 3.500
}

res = [512, 128, 256]  # (nx, ny, nz)

lith_vol = lith_vol.reshape(res)  # (X, Y, Z)

velocity_vol = np.vectorize(lit_to_vp.get)(lith_vol)  # → también será (X, Y, Z)

y_index = res[1] // 2  # 36

corte_y = velocity_vol[:, y_index, :].T  # (Z, X)

x_coords = np.linspace(1145780, 1156020, 512) 
z_coords = np.linspace(720, 3280, 256)       

mask_bajo_topo = ~geo_data.grid.topography.topography_mask

mask_bajo_topo_3d = mask_bajo_topo.reshape(lith_vol.shape)

velocity_vol_masked = np.where(mask_bajo_topo_3d, velocity_vol, np.nan)

y_index = lith_vol.shape[1] // 2
corte_y = velocity_vol_masked[:, y_index, :].T

plt.figure(figsize=(12, 6))
im = plt.imshow(corte_y, cmap='jet', origin='lower',
                extent=[1145780, 1156020, 720, 3280],
                aspect='auto',
                vmin=1.800, vmax=4.000)
plt.colorbar(im, label='Vp (m/s)')
plt.xlabel('X (m)')
plt.ylabel('Z (m)')
plt.tight_layout()
plt.show()

vp_corr = np.flip(velocity_vol_masked, axis=2)
vp = np.asarray(vp_corr)
vp = np.nan_to_num(vp, nan=1.8, posinf=None, neginf=None)

xmin, xmax = 1145780, 1156020
ymin, ymax = 1164620, 1167180
zmin, zmax = 720, 3280

nx, ny, nz = 512, 128, 256
dx, dy, dz = 20.0, 20.0, 10.0
origin = (1145780.0, 1164620.0, 720.0)   # (xmin, ymin, zmin)
shape  = (nx, ny, nz)
spacing = (dx, dy, dz)

seis_model = Model(
    vp=vp,
    origin=origin,
    spacing=spacing,
    shape=shape,
    nbl=30,
    space_order=4,
    bcs="damp"
)

t0 = 0. 
tn = 4096. 
dt = 1.0

time_range = TimeAxis(start=t0, stop=tn, step=dt)

f0 = 0.008
src = RickerSource(name='src', grid=seis_model.grid, f0=f0,
                   npoint=1, time_range=time_range)

src.coordinates.data[:] = seis_model.origin + np.array(seis_model.domain_size) * .5
src.coordinates.data[0, -1] = seis_model.origin[2] + 20

rec = Receiver(name='rec', grid=seis_model.grid,
               npoint=512, time_range=time_range)

rec.coordinates.data[:, 0] = seis_model.origin[0] + np.linspace(0, seis_model.domain_size[0], num=512)
rec.coordinates.data[:, 1] = seis_model.origin[1] + 0.5 * seis_model.domain_size[1]
rec.coordinates.data[:, -1] = seis_model.origin[2] + 20

u = dv.TimeFunction(name="u", grid=seis_model.grid, time_order=2, space_order=4)
u.data.shape
u.shape == u.data.shape
isinstance(u.data, np.ndarray)
u.dx2.evaluate
pde = seis_model.m * u.dt2 - u.laplace + seis_model.damp * u.dt
u.forward
u.backward
u.forward.dx
stencil = dv.Eq(u.forward, dv.solve(pde, u.forward))
src_term = src.inject(field=u.forward, expr=src * dt**2 / seis_model.m)
rec_term = rec.interpolate(expr=u.forward)

op = dv.Operator([stencil] + src_term + rec_term, subs=seis_model.spacing_map)
op(time=time_range.num-1, dt=seis_model.critical_dt)

G = rec.data.astype(np.float32)      
nt, nrec = G.shape

dt_us = int(round(float(time_range.step) * 1000.0))

xs = rec.coordinates.data[:, 0].astype(np.int32)
ys = rec.coordinates.data[:, 1].astype(np.int32)
x0 = int(src.coordinates.data[0, 0])
y0 = int(src.coordinates.data[0, 1])

traces = []
for i in range(nrec):
    tr = Trace(G[:, i])
    tr.stats.delta = dt_us / 1_000_000.0  
    tr.stats.segy = {'trace_header': {
        'source_coordinate_x': x0,
        'source_coordinate_y': y0,
        'group_coordinate_x':  int(xs[i]),
        'group_coordinate_y':  int(ys[i]),
        'coordinate_units':    1,    
        'scalar_to_be_applied_to_all_coordinates': 1, 
        'trace_identification_code': 1, 
        'trace_sequence_number_within_line': i+1,
        'ensemble_number': 1,           
        'trace_number_within_the_ensemble': i+1,
        'delay_recording_time': 0,   
    }}
    traces.append(tr)

st = Stream(traces)

def write_segy(path, G, dt_ms, sx, sy, xs, ys, shot_id, line_id):
    """
    Escribe un gather (nt, ntr) a SEG-Y IEEE float32.
    dt_ms: paso de muestreo en milisegundos (coherente con tu TimeAxis).
    sx, sy: coords de la fuente (int, en metros).
    xs, ys: arrays (ntr,) con coords de receptores (int, en metros).
    shot_id, line_id: IDs para los headers.
    """
    G = np.asarray(G, dtype=np.float32, order="C")
    nt, ntr = G.shape
    dt_us = int(round(float(dt_ms) * 1000.0))

    traces = []
    for i in range(ntr):
        tr = Trace(G[:, i])
        tr.stats.delta = dt_us / 1_000_000.0
        tr.stats.segy = {'trace_header': {
            'source_coordinate_x': int(sx),
            'source_coordinate_y': int(sy),
            'group_coordinate_x':  int(xs[i]),
            'group_coordinate_y':  int(ys[i]),
            'coordinate_units':    1,  
            'scalar_to_be_applied_to_all_coordinates': 1, 
            'trace_identification_code': 1,              
            'trace_sequence_number_within_line': i+1,
            'ensemble_number': int(1000*line_id + shot_id),
            'trace_number_within_the_ensemble': i+1,
            'delay_recording_time': 0,
        }}
        traces.append(tr)

    Stream(traces).write(path, format="SEGY", data_encoding=5)
    
t0 = 0.0
tn = 4096.0
dt_ms = 1.0
time_range = TimeAxis(start=t0, stop=tn, step=dt_ms)
dt = time_range.step

x0, y0, z0 = map(float, seis_model.origin)         
Lx, Ly, Lz = map(float, seis_model.domain_size)     
dx, dy, dz = map(float, seis_model.spacing)    

Ns = 64
sx_all = x0 + np.linspace(0.0, Lx, Ns)
y_lines = y0 + np.array([0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]) * Ly
sz = z0 + 20.0

xs_base = rec.coordinates.data[:, 0].copy().astype(np.int32)

outdir = "shot densos 2"
os.makedirs(outdir, exist_ok=True)

for line_id, sy in enumerate(y_lines, start=1):
    print(f"\n=== Línea Y {line_id}: Y = {sy:.2f} m ===")

    rec.coordinates.data[:, 1] = float(sy)
    rec.coordinates.data[:, 2] = float(sz)
    ys_rec = rec.coordinates.data[:, 1].astype(np.int32)
    xs_rec = xs_base 

    for shot_id, sx in enumerate(sx_all, start=1):
        src.coordinates.data[0, 0] = float(sx)
        src.coordinates.data[0, 1] = float(sy)
        src.coordinates.data[0, 2] = float(sz)

        u.data[:] = 0.0
        rec.data[:] = 0.0
        op(time=time_range.num-1, dt=dt)

        fname = os.path.join(outdir, f"shot_y{line_id:02d}_{shot_id:03d}.segy")
        write_segy(fname, rec.data, dt_ms, sx=int(sx), sy=int(sy),
                   xs=xs_rec, ys=ys_rec, shot_id=shot_id, line_id=line_id)

        if shot_id in (1,2,3) or (shot_id % 10 == 0) or (shot_id == Ns):
            print(f"[L{line_id:02d} S{shot_id:03d}] "
                  f"SRC=({sx:.2f}, {sy:.2f}, {sz:.2f}) | "
                  f"REC X[min,max]=({xs_rec.min()}, {xs_rec.max()}) "
                  f"Y={ys_rec[0]} Z={int(rec.coordinates.data[0,2])} "
                  f"-> {os.path.basename(fname)}")

carpeta = "shot densos 2"
for segy_path in sorted(glob.glob(os.path.join(carpeta, "*.segy"))):
    try:
        st = read(segy_path, format="SEGY")
        nrec = len(st)
        nt = st[0].stats.npts
        dt_ms = st[0].stats.delta * 1000.0
        tmax_ms = (nt-1) * dt_ms
        G = np.column_stack([tr.data for tr in st]).astype(np.float32)

        v = np.percentile(np.abs(G), 99.5); v = float(v if v>0 else (np.max(np.abs(G)) or 1.0))

        plt.figure(figsize=(16,8))
        plt.imshow(G, cmap="gray", aspect="auto",
                   extent=(0, nrec-1, tmax_ms, 0.0),
                   vmin=-0.01, vmax=0.01)
        plt.xlabel("Trace #"); plt.ylabel("Time (ms)")
        plt.colorbar(); plt.tight_layout()

        png_path = os.path.splitext(segy_path)[0] + ".png"
        plt.savefig(png_path, dpi=150); plt.close()
        print("✅", os.path.basename(png_path))
    except Exception as e:
        print("⚠️ No pude plotear:", segy_path, "|", e)

carpeta = "shot densos 2"
files = sorted(glob.glob(os.path.join(carpeta, "*.segy")))
if not files:
    raise FileNotFoundError(f"No encontré archivos .segy en: {carpeta}")

src_set = set()
rec_set = set()

def get_hdr(tr, key, default=np.nan):
    return tr.stats.segy.trace_header.get(key, default)

for segy_path in files:
    st = read(segy_path, format="SEGY", headonly=True)
    for tr in st:
        gx = get_hdr(tr, "group_coordinate_x")
        gy = get_hdr(tr, "group_coordinate_y")
        if np.isfinite(gx) and np.isfinite(gy):
            rec_set.add((int(gx), int(gy)))
        sx = get_hdr(tr, "source_coordinate_x")
        sy = get_hdr(tr, "source_coordinate_y")
        if np.isfinite(sx) and np.isfinite(sy):
            src_set.add((int(sx), int(sy)))

recs = np.array(sorted(rec_set), dtype=float) if rec_set else np.empty((0,2))
srcs = np.array(sorted(src_set), dtype=float) if src_set else np.empty((0,2))

print(f"Receptores únicos: {recs.shape[0]}  |  Fuentes únicas: {srcs.shape[0]}")
if recs.size == 0 and srcs.size == 0:
    raise RuntimeError("No pude extraer coordenadas de headers SEG-Y (revisa 'coordinate_units' y 'scalar').")

dx, dy, dz = map(float, seis_model.spacing)
nbl = int(seis_model.nbl)

x0 = float(seis_model.origin[0])
y0 = float(seis_model.origin[1])
x1 = x0 + float(seis_model.domain_size[0])
y1 = y0 + float(seis_model.domain_size[1])

x0_ext = x0 - nbl * dx
y0_ext = y0 - nbl * dy
x1_ext = x1 + nbl * dx
y1_ext = y1 + nbl * dy

fig, ax = plt.subplots(figsize=(20, 8))

ax.add_patch(Rectangle((x0_ext, y0_ext), (x0 - x0_ext), (y1_ext - y0_ext),
                       facecolor="0.85", edgecolor="0.85"))
ax.add_patch(Rectangle((x1, y0_ext), (x1_ext - x1), (y1_ext - y0_ext),
                       facecolor="0.85", edgecolor="0.85"))
ax.add_patch(Rectangle((x0, y0_ext), (x1 - x0), (y0 - y0_ext),
                       facecolor="0.85", edgecolor="0.85"))
ax.add_patch(Rectangle((x0, y1), (x1 - x0), (y1_ext - y1),
                       facecolor="0.85", edgecolor="0.85"))

ax.add_patch(Rectangle((x0, y0), (x1 - x0), (y1 - y0),
                       fill=False, linewidth=1.5, edgecolor="black", label="Dominio físico"))

if recs.size:
    ax.scatter(recs[:,0], recs[:,1], s=5, c='blue', alpha=0.6, label='Receptores')
if srcs.size:
    ax.scatter(srcs[:,0], srcs[:,1], s=28, facecolors='none', edgecolors='r', linewidths=1.2, label='Fuentes')

padx = 0.02 * (x1_ext - x0_ext)
pady = 0.02 * (y1_ext - y0_ext)
ax.set_xlim(x0_ext - padx, x1_ext + padx)
ax.set_ylim(y0_ext - pady, y1_ext + pady)
ax.set_aspect('equal', adjustable='box')

halo_handle = Patch(facecolor="0.85", edgecolor="0.6", label="Halo (PML)")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles + [halo_handle], labels + ["Halo (PML)"],
          loc="upper left", bbox_to_anchor=(1.02, 1), borderaxespad=0.)

ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_title("Ubicación de fuentes y receptores (desde 'shot densos')")
ax.grid(True, linestyle=":")

fig.subplots_adjust(right=0.80)
plt.show()
